Настя Скутина и Юля Маркова (БКЛ182)

In [383]:
import requests
session = requests.session()

import re
import time
import random
from bs4 import BeautifulSoup

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)


In [384]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
morph_vocab = MorphVocab()

In [6]:
link = 'https://stihi.ru/poems/list.html?topic=all&type=selected&year=2020&month=01&day=1'

In [8]:
month_links = []
for x in range (1,11):
    if x < 10:
        n = '0' + str(x)
    else:
        n = str(x)
    link_n = re.sub(r'&month=(\d\d)&', '&month='+n+'&', link)
    month_links.append(link_n)

In [9]:
def get_links(link):
    poem_links = []
    page = session.get(link, headers={'User-Agent': ua.random}).text
    soup = BeautifulSoup(page, 'html.parser')
    for a in soup.find_all('a', {'class': 'poemlink'}):
        raw_link = re.search(r'href="(.*)">', str(a))
        poem_links.append('https://stihi.ru' + raw_link.group(1))
    clear_links = poem_links[:15] + poem_links[25:]
    return clear_links

In [10]:
all_links = []
for month in month_links:
    all_links.extend(get_links(month))

In [12]:
def get_poems(link):
        poems_dict = {}
        page = session.get(link, headers={'User-Agent': ua.random}).text
        soup = BeautifulSoup(page, 'html.parser')
        poems_dict['Text'] = soup.find('div', {'class': 'text'}).text
        poems_dict['Title'] = soup.find('h1').text
        poems_dict['Author'] = soup.find('div', {'class': 'titleauthor'}).text
        return poems_dict
    

In [15]:
texts = []
for link in all_links:
    time.sleep(random.uniform(3.1, 7.2))
    texts.append(get_poems(link))

In [20]:
text_texts = []
for text in texts:
    text_texts.extend(text['Text'].split())
len(text_texts) #показывает количество вхождений в корпусе

17117

In [24]:
LEMM_DICTIONARY = {}

In [25]:
def lemm_dict(lemm, idx):
    if lemm in LEMM_DICTIONARY:
        if idx not in LEMM_DICTIONARY[lemm]:
            LEMM_DICTIONARY[lemm].append(idx)
    else:
        LEMM_DICTIONARY[lemm] = [idx]

In [173]:
def parse_sentence(sent, idx):
    dic = {}
    dic['raw_sent'] = sent
    dic['lemm_sent'] = []
    dic['meta_word'] = []
    dic['pos_sent'] = []
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    lemm_list = []
    
    for toc in doc.tokens:
        toc.lemmatize(morph_vocab)
        if toc.pos != 'PUNCT':
            pos = toc.pos
            word = toc.text.lower()
            lemm = toc.lemma
            lemm_dict(lemm, idx)
            dic['lemm_sent'].append(lemm)
            dic['meta_word'].append((word, pos))
            dic['pos_sent'].append(pos)
    return dic

In [61]:
def text_processing(text_info, idx):
    dic = {}
    dic['Author'] = text_info['Author']
    dic['Title'] = text_info['Title']
    dic['TextwInfo'] = []
    doc = Doc(text_info['Text'])
    doc.segment(segmenter)
    for sent in doc.sents:
        dic['TextwInfo'].append(parse_sentence(sent.text, idx))
    return dic

In [174]:
CORPUS = []
for idx, text in enumerate(texts):
    processed_text = text_processing(text, idx)
    CORPUS.append(processed_text)

In [320]:
def word2lemm(word):
    lemm_list = []
    for item in morph.parse(word):
        if item.normal_form not in lemm_list:
            new_item = re.sub('ё', 'е', item.normal_form)
            lemm_list.append(new_item)
    return lemm_list

In [321]:
def lemm_search(word):
    lemms = word2lemm(word)
    lemm2text = {}
    for lemm in lemms:
        if lemm in LEMM_DICTIONARY:
            lemm2text[lemm] = LEMM_DICTIONARY[lemm]
    return lemm2text

In [135]:
def if_wordform(sent, wordform, lemm):
    word_idx = sent['lemm_sent'].index(lemm)
    if wordform in sent['meta_word'][word_idx]:
        return 1
    else:
        return 0

In [136]:
def if_pos(sent, pos, lemm):
    word_idx = sent['lemm_sent'].index(lemm)
    if pos in sent['meta_word'][word_idx]:
        return 1
    else:
        return 0

In [137]:
def word_in_text(lemm, text_idx, wordform=None, pos=None):
    TextwInfo = CORPUS[text_idx]['TextwInfo']
    sent_list= []
    for sent in TextwInfo:
            if lemm in sent['lemm_sent']:
                if wordform == None and pos == None:
                    sent_list.append(sent['raw_sent'])
                    
                elif wordform == None and pos != None:
                    if if_pos(sent, pos, lemm) == 1:
                        sent_list.append(sent['raw_sent'])
                        
                elif wordform != None and pos == None:
                    if if_wordform(sent, wordform, lemm) == 1:
                        sent_list.append(sent['raw_sent'])
                
                else:
                    if if_wordform(sent, wordform, lemm) == 1:
                        if if_pos(sent, pos, lemm) == 1:
                            sent_list.append(sent['raw_sent'])

    return (sent_list, CORPUS[text_idx]['Title'], CORPUS[text_idx]['Author'])

In [195]:
def parse1word(request):
    wordform = re.search(r'"(\w*)"', request)
    pos = re.search(r'\+(\w*)', request)
    if wordform == None:
        if pos == None:
            word = request
        else:
            pos = pos.group(1)
            word = re.search(r'(\w*)\+', request).group(1)
    else:
        wordform = wordform.group(1)
        word = wordform
        if pos != None:
            pos = pos.group(1)
    
    return word, wordform, pos

In [385]:
def search_1word(request):
    word, wordform, pos = parse1word(request)
    LemmwIdx = lemm_search(word)
    if len(LemmwIdx) != 0:
        for item in LemmwIdx:
            lemm = item
            none_check = 0
            results = []
            for text_idx in LemmwIdx[item]:
                sentence = word_in_text(lemm, text_idx, wordform=wordform, pos=pos)
                if len(sentence[0]) != 0:
                    none_check = 1
                    results.append(sentence)
            if none_check == 0:
                print('Не нашлось :(')
            else:
                return results

    else:
        print('Не нашлось :(')

In [198]:
def search_pos(request):
    results = []
    for text in CORPUS:
        suit_sents = []
        for sent in text['TextwInfo']:
            if request in ' '.join(sent['pos_sent']):
                suit_sents.append(sent['raw_sent'])
        if len(suit_sents) != 0:
            results.append((suit_sents, text['Title'], text['Author']))
    if len(results) != 0:
        return results
    else:
        print('sorry there is no such pos sequence in corpus :(')

In [231]:
def parse_word_in_ngrm(req_word):
    wordform = re.search(r'"(\w*)"', req_word)
    pos = re.search(r'[A-Z]+', req_word)
    if wordform == None:
        if pos == None:
            word = req_word
        else:
            pos = pos.group(0)
            word = re.search(r'(\w*)\+', req_word)
            if word != None:
                word = word.group(1)
    else:
        wordform = wordform.group(1)
        word = wordform
        if pos != None:
            pos = pos.group(0)
    
    return word, wordform, pos

In [235]:
def parse_ngrm(request):
    words = request.split()
    parsed_words = []
    for word in words:
        parsed_word = parse_word_in_ngrm(word)
        parsed_words.append(parsed_word)
    return parsed_words

In [244]:
def get_sents(word1, word2, n=1):
    results = []
    for lemm in word1['lemms']:
        for text_idx in word1['lemms'][lemm]:
            sent_list = []
            for sent in CORPUS[text_idx]['TextwInfo']:
                if lemm in sent['lemm_sent']:
                    if check_word(sent, word1, sent['lemm_sent'].index(lemm)) == 1:
                        word_idx = sent['lemm_sent'].index(lemm) + n
                        if check_word(sent, word2, word_idx) == 1:
                            sent_list.append(sent['raw_sent'])
            if len(sent_list) != 0:
                results.append((sent_list, CORPUS[text_idx]['Title'], CORPUS[text_idx]['Author']))
        return results

In [277]:
def check_word(sent, word_info, word_idx):
    lemms = word_info['lemms']
    wordform = word_info['wordform']
    pos = word_info['pos']
    check = 0
    if -1 < word_idx < len(sent['lemm_sent']):
        if lemms == None or sent['lemm_sent'][word_idx] in lemms:
            if wordform == None or wordform == sent['meta_word'][word_idx][0]:
                if pos == None or pos == sent['meta_word'][word_idx][1]:
                    check = 1
    return check

In [386]:
def search_bgrm(request):
    bgrm = parse_ngrm(request)
    words_inf = []
    for word in bgrm:
        word_inf = {}
        word_inf['wordform'] = word[1]
        word_inf['pos'] = word[2]
        if word[0]:
            word_inf['lemms'] = lemm_search(word[0])
        else:
            word_inf['lemms'] = None
        words_inf.append(word_inf)
    if words_inf[0]['lemms'] != None:
        results = get_sents(words_inf[0], words_inf[1], 1)
    else:
        results = get_sents(words_inf[1], words_inf[0], -1)
    if len(results) != 0:
        return results
    else:
        print('Не нашлось :(')

In [377]:
def the_search(request):
    print('\n\n')
    if ' ' not in request:
        results = search_1word(request)
    else:
        if request == request.upper():
            results = search_pos(request)
        else:
            results = search_bgrm(request)
    if results != None:
        for result in results:
            for sent in result[0]:
                print('"'+result[1]+'" -', result[2])
                print(sent, '\n')

In [390]:
print('Введите слово или словосочетание:')
request = input()
the_search(request)

Введите слово или словосочетание:
смерть



"Рубище" - Владислав Савенко
Этим ветрам насвистывать
Музыку горних сфер,
Что словно смерть неистова,
Оголена  как нерв. 

"В дорогу" - Екатерина Кычкина
Я люблю и буду любить всегда,
Даже когда стану совсем седа,
Ведь любовь - за рамками круговерти,
Неподвластна времени, как и смерти,
Она свет, разлитый в моей груди,
Говорящий: "Всё хорошо, иди". 

"Ветер на планете" - Мария Махова
Ты видишь дни, ты видишь всё на свете, ты слышишь эту музыку вдали, весь этот путь от жизни и до смерти, весь этот стон и дым вокруг земли, а кто нас встретит, где нас нынче встретят, покуда страх нам душу не сковал… 

"А мы с тобой проснёмся поутру" - Юлия Долгановских
а мы с тобой проснёмся поутру
агу и ты ответишь мне агу
врагу покажется мы потеряли разум
а мы с тобою только родились
и пеленой беспомощная жизнь
опутывает нас обоих разом

агу прости а где а кто а я
а ты а мы и в полнокровный полдень
отряхивая солнце с мутных глаз
мы оба ночь рождения не вспомним